In [13]:
from itertools import product

import os
import pandas as pd


base_path = "/scratch/gpfs/jp7427/out/isotropic-decaying-turbulence/128x128/cupy/"

In [14]:
# Loop parameters
M_max_values = [0.01, 0.1, 1, 10, 40, 50]

seeds = range(1, 31)

configs = [
    "MUSCL-Hancock",
    "ZS3",
    "ZS7",
    "ZS3t",
    "ZS7t",
    "MM3/1rev/rtol_1e-3",
    "MM3/1rev/rtol_1e-2",
    "MM7/1rev/rtol_1e-3",
    "MM7/1rev/rtol_1e-2",
    "MM3/2revs/rtol_1e-3",
    "MM3/2revs/rtol_1e-2",
    "MM7/2revs/rtol_1e-3",
    "MM7/2revs/rtol_1e-2",
    "MM3/3revs/rtol_1e-3",
    "MM3/3revs/rtol_1e-2",
    "MM3/3revs/rtol_1e-1",
    "MM3/3revs/rtol_1e0",
    "MM3/3revs/rtol_1e1",
    "MM3/3revs/no_NAD",
    "MM7/3revs/rtol_1e-3",
    "MM7/3revs/rtol_1e-2",
    "MM7/3revs/rtol_1e-1",
    "MM7/3revs/rtol_1e0",
    "MM7/3revs/rtol_1e1",
    "MM7/3revs/no_NAD",
]

# read simulation outputs
data = []
for config, M_max in product(configs, M_max_values):
    entry = dict(config=config, M_max=M_max, failed=0)
    for seed in seeds:
        if M_max < 1 and seed > 1:
            continue

        sim_path = f"{base_path}{config}/M_max_{M_max}/seed_{seed:02d}/"

        if not os.path.exists(sim_path):
            raise FileNotFoundError(sim_path)
        elif os.path.exists(sim_path + "error.txt"):
            entry["failed"] += 1
        elif os.path.exists(sim_path + "timings.txt"):
            continue
        else:
            raise FileNotFoundError(f"error.txt or timings.txt in {sim_path}")
    data.append(entry)

df = pd.DataFrame(data)

In [15]:
df.pivot(index="config", columns="M_max", values="failed").loc[configs, :]

M_max,0.01,0.10,1.00,10.00,40.00,50.00
config,,,,,,
MUSCL-Hancock,0,0,0,0,0,0
ZS3,0,0,0,0,0,0
ZS7,0,0,0,0,0,0
ZS3t,0,0,0,30,30,30
ZS7t,0,0,0,30,30,30
MM3/1rev/rtol_1e-3,0,0,0,30,30,30
MM3/1rev/rtol_1e-2,0,0,0,30,30,30
MM7/1rev/rtol_1e-3,0,0,0,29,30,30
MM7/1rev/rtol_1e-2,0,0,0,30,30,30


In [4]:
df_to_table = df[df.M_max >= 1]


df_to_table["config"] = pd.Categorical(
    df_to_table["config"], categories=configs, ordered=True
)

print(df_to_table.pivot(index="config", columns="M_max", values="failed").to_latex())

\begin{tabular}{lrrrrrr}
\toprule
M_max & 1.000000 & 10.000000 & 20.000000 & 30.000000 & 40.000000 & 50.000000 \\
config &  &  &  &  &  &  \\
\midrule
p0 & 0 & 0 & 0 & 0 & 0 & 0 \\
MUSCL-Hancock & 0 & 0 & 0 & 0 & 0 & 0 \\
ZS3 & 0 & 0 & 0 & 0 & 0 & 0 \\
ZS7 & 0 & 0 & 0 & 0 & 0 & 0 \\
ZS3t & 0 & 30 & 30 & 30 & 30 & 30 \\
ZS7t & 0 & 30 & 30 & 30 & 30 & 30 \\
MM3-3/rtol_1e-2_atol_1e-14 & 0 & 0 & 0 & 0 & 0 & 0 \\
MM7-3/rtol_1e-2_atol_1e-14 & 0 & 0 & 0 & 0 & 0 & 0 \\
MM3-3/rtol_1e-1_atol_1e-14 & 0 & 0 & 0 & 0 & 0 & 0 \\
MM7-3/rtol_1e-1_atol_1e-14 & 0 & 0 & 0 & 0 & 0 & 0 \\
MM3-3/rtol_1e0_atol_1e-14 & 0 & 30 & 30 & 30 & 30 & 30 \\
MM7-3/rtol_1e0_atol_1e-14 & 1 & 30 & 30 & 30 & 30 & 30 \\
MM3-3/rtol_1e1_atol_1e-14 & 11 & 30 & 30 & 30 & 30 & 30 \\
MM7-3/rtol_1e1_atol_1e-14 & 29 & 30 & 30 & 30 & 30 & 30 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1057696/1095294678.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_table["config"] = pd.Categorical(
